# ISE 447 - Financial Optimization - HW 2

## Jack Dean
## Lehigh University

Due Date: Monday March 21st, 2022  
Total: 100 Points  
*Note*: Under otherwise stated, all textbook references are for “Optimization Methods in Finance”

In [ ]:
'''Packages used in this work'''
import numpy as np
import pandas as pd
import scipy.stats as stats
import pulp
from pulp import *
import matplotlib.pyplot as plt
import seaborn as sns

## Problem 1 -- Reformulating Linear Programs & Duality (10 Points)

Consider the modified version of the fund allocation problem considered in HW 1, Question 5.  Recall that a linear program in standard form is
$$\min c^Tx$$
$$\mathrm{s.t.}\; Ax=b$$
$$x \geq 0$$
First reformulate the modified version of the fund allocation problem considere din HW 1, Question 5, in standard form.  Then find the linear programming dual associated to it.

### RECALL: Modified Fund Allocation Problem (as in HQ 1 Solutions)

$$\max 0.10f_1 + 0.15f_2 + 0.16f_3 + 0.08f_4$$
$$\mathrm{s.t.}\;\;\; f_1 + f_2 + f_3 + f_4 + s = 80,000$$
$$0.50f_1 + 0.30f_2 + 0.25f_3 + 0.60f_4 \geq 0.35 * (f_1 + f_2 + f_3 + f_4)$$
$$0.30f_1 + 0.10f_2 + 0.40f_3 + 0.20f_4 \geq 0.30 * (f_1 + f_2 + f_3 + f_4)$$
$$0.20f_1 + 0.60f_2 + 0.35f_3 + 0.20f_4 \geq 0.15 * (f_1 + f_2 + f_3 + f_4)$$
$$s - \alpha0.50f_1 - \alpha0.30f_2 - \alpha0.25f_3 - \alpha0.60f_4 \geq 0$$
$$$$
$$f_1,f_2,f_3,f_4,s \geq 0$$

### Solution

## Problem 2 -- Linear Programming Duality (Based on Exercise 2.3 in Textbook) (10 Points)

Consider a Linear Program of the Form:
$$\max c^Tx$$
$$\mathrm{s.t.} \; Ax\leq b$$
$$x \geq 0$$
where
$$A = \begin{pmatrix}
1 & 1 & 1 & 1 & 1\\
5 & 4 & 3 & 2 & 1\\
\end{pmatrix},\;\;\; b = \begin{pmatrix}
3\\
14\\
\end{pmatrix},\;\;\;
c=\begin{pmatrix}
6\\
5\\
4\\
3\\
2\\
1\\
\end{pmatrix}$$
First, obtain the dual of this problem. Now solve the dual problem that you obtain using your favorite solver. Does the objective of the dual LP tells you something about the objective value of the original problem?


*Hint*: notice that the problem is in “dual form”, so in a sense you are really looking for the primal associated to this dual  
*Note*: because it has only two variables, you could also solve the problem by the graphical method

### Solution

In [ ]:
'''Code Block'''

## Problem 3 -- Dedication and Immunization portfolios (Based on Excercise 3.1 in the Textbook 3.1) (50 Points)

You need to create a portfolio to cover the following stream of liabilities for the next six future periods:

| Period | 1 | 2 | 3 | 4 | 5 | 6 |
| -- | -- | -- | -- | -- | -- | -- |
| Liability | 500 | 200 | 800 | 200 | 800 | 1200 |

For that purpose, you may purchase the bonds in Table 1.

The term structure of risk-free insterest rate is (to be used for Immunization Constraints)

| Period | 1 | 2 | 3 | 4 | 5 | 6 |
| -- | -- | -- | -- | -- | -- | -- |
| Rate | 5.04% | 5.94% | 6.36% | 7.18% | 7.89% | 8.39% |

### Part (a) -- Dedication (10 Points)
Formulate a linear programming model to find the lowest-cost long-only (only buy bonds) dedicated portfolio that covers the stream of liabilities with the bonds above. Assume that surplus balances can be carried from one period to the next but earn no interest. What is the cost of your portfolio? What is the composition of your portfolio?

*Table 1*
| Bond | 1 | 2 | 3 | 4 | 5 | 6 | Price |
| -- | -- | -- | -- | -- | -- | -- | -- |
| 1 | 10 | 10 | 10 | 10 | 10 | 110 | 109 |
| 2 | 7 | 7 | 7 | 7 | 7 | 107 | 94.8 |
| 3 | 8 | 8 | 8 | 8 | 8 | 108 | 99.5 |
| 4 | 6 | 6 | 6 | 6 | 106 | 0 | 93.1 |
| 5 | 7 | 7 | 7 | 7 | 107 | 0 | 97.2 |
| 6 | 6 | 6 | 6 | 106 | 0 | 0 | 93.1 |
| 7 | 5 | 5 | 5 | 105 | 0 | 0 | 97.2 |
| 8 | 10 | 10 | 110 | 0 | 0 | 0 | 110 |
| 9 | 8 | 8 | 108 | 0 | 0 | 0 | 104 |
| 10 | 6 | 6 | 106 | 0 | 0 | 0 | 101 |
| 11 | 10 | 110 | 0 | 0 | 0 | 0 | 107 |
| 12 | 7 | 107 | 0 | 0 | 0 | 0 | 102 |
| 13 | 100 | 0 | 0 | 0 | 0 | 0 | 95.2 |

### Part (b) -- Term Structure (5 Points)
Use the linear programming sensitivity information obtained by solving the problem in Part (a) to determine the implied term structure of interest rates

### Part (c) -- Overpriced Bonds (5 Points)
Using sensitivity analysis information, and in particular the reduced costs associated to the different bonds, can you identify bonds that are overpriced? By how much? Briefly explain.

### Part (d) -- Term Structure cont. (5 Points)
Suppose the streams of liabilities changes to

| Period | 1 | 2 | 3 | 4 | 5 | 6 |
| -- | -- | -- | -- | -- | -- | -- |
| Liability | 500 | 200 | 800 | 200 | 800 | 1200 |

Find the new optimal dedicated portfolio and determine its associated new implied term structure. Is it different from the one you obtained in Part (b)? Can you provide an intuitive explanation for the difference or lack of thereof?

### Part (e) -- Immunization (25 Points)
Using the problem's original data:

#### Sub-Part (i) -- (10 Points)
Find the lowest cost portfolio that matches the present value and duration of the stream of liabilities.

#### Sub-Part (ii) -- (5 Points)
Compare the costs and the cash flow streams vs liabilities of both the Dedication and Immunized portfolio. Discuss the trade-offs of each solution. Briefly explain which portfolio would you prefer.

#### Sub-Part (iii) -- (10 Points)
How does the present value of the Immunized portfolio change if the current rates decrease (all) by 1 percentage point? How does the present value of the Immunized portfolio change if the current rates increase (all) by 1 percentage point? How does the present value of the Immunized portfolio change if the current rates of periods 1, 2 decrease by 1%, of periods 3, 4, 5 increase by 1%, and of period 6 decreases by 1%. Can you draw any conclusions from your results.


### Solution to Part (a)

In [ ]:
'''Code Block'''

### Solution to Part (b)

In [ ]:
'''Code Block'''

### Solution to Part (c)

In [ ]:
'''Code Block'''

### Solution to Part (d)

In [ ]:
'''Code Block'''

### Solution to Part (e)

#### Solution to Sub Part (i)

In [ ]:
'''Code Block'''

#### Solution to Sub Part (ii)

In [ ]:
'''Code Block'''

#### Solution to Sub Part (iii)

In [ ]:
'''Code Block'''

## Problem 4 -- Dedication with Bond Ratings (Based on Exercise 3.1 in the Textbook) (10 Points)
Consider basically the same instance of the Dedication portfolio considered in the previous question. However, now the prices of the bonds are a bit different, and for each bond, we are giving his risk rating (think of Moody’s or Bloomberg ratings).

| Bond | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 |
| -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- |
| Price | 108 | 94 | 99 | 92.7 | 96.6 | 95.9 | 92.9 | 100 | 104 | 101 | 107 | 102 | 95.2 |
| Rating | B | B | B | B | B | B | A | A | A | A | A | A | A |

Solve the Dedication portfolio problem of the previous question with the new prices, but this time assume that at most 50% of the money that you invest in Bonds should be allocated to “B” rated Bonds. What is the cost of your portfolio? What is the composition of your portfolio.

### Solution

In [ ]:
'''Code Block'''

## Problem 4 -- Other Liability Management Problem (Based on Exercise 3.4/3.5 in the Textbook) (10 Points)
A company will face the following cash requirement in the next eight quarters (positive numbers indicate cash needs while negative entries represent cash surpluses):

| Period | Q1 | Q2 | Q3 | Q4 | Q5 | Q6 | Q7 | Q8 |
| -- | -- | -- | -- | -- | -- | -- | -- | -- |
| Liability/Surplus | 100 | 500 | 100 | -600 | -500 | 200 | 600 | -900 |

To cover cash needs, the company has three borrowing possibilities:  
* A 2-year loan available at the beginning of Q1, with a 1% interest rate per quarter  
* The other two borrowing opportunities are available at the beginning of every quarter: a 6-month loan with 1.8% interest rate per quarter, and a quarterly loan with 2.5% interest per quarter  
* Any surplus can be invested at a 0.5% interest rate per quarter

### Part (a) -- (5 Points)
Formulate a linear program that maximizes the wealth of the company at the beginning of Q9

### Part (b) -- (5 Points)
Using your favorite solver, solve the liability management problem above, and find out the dual values associated to each of the balance per quarter constraints. Use the latter information to solve the following question: Supose that the cash requirement in Q2 is 300 (instead of 500). Use the shadow price associated to the Q2 balance constraint to find out by how much would the wealth in Q9 would change.


### Solution to Part (a)

In [ ]:
'''Code Block'''

### Solution to Part (b)

In [ ]:
'''Code Block'''

## Problem 6 -- Home Financing Problem (Based on Exercise 3.6 in the Textbook) (10 Points)
A home buyer can combine several mortgage loans to finance the purchase of a house. Regulations impose limits on the amount that can be borrowed from certain sources as well as a limit on the total reimbursement each
month. Let $B$ be the borrowing needs at $T$ the nmber of months over which the loans will be paid back. There are $n$ different loan opportunities available. Loan $i$ has a fixed interest rate $t_i$, and length $T_i ≤ T$ and a maximum amount borrowed $b_i$. The monthly payment $m_i$ is required each month. Furthermore,
we would like the total monthly payment $p$ over all loans to be constant. Formulate a linear program that
finds a combination of the loans that minimizes the home buyer’s cost of borrowing.

*Hint*: In addition to variables $x_t_i$ for the the payment of loan $i$ in month $t$, it may be useful to introduce a variable for the amount of outstanding principal on loan $i$ in month $t$.


### Solution

In [ ]:
'''Code Block'''